In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import json
import copy
import json
from tqdm import tqdm
import pickle
import logging
import math
import matplotlib
# plot calibration of Copilot confidences with XGBoost predictions
from re import S
from scipy.stats.stats import pearsonr  
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
plt.rc('text', usetex=True)
plt.rc('font', family='sans-serif')

In [ ]:

with open('../data/data_labeled_study.pkl', 'rb') as f:
    logs_by_user_session_labeled = pickle.load(f)


In [ ]:
df_survey = pd.read_csv('../data/copilot_post_survey.csv')

# Participants Details (Section 5.2)

In [ ]:
# create bar chart for each question
from tkinter import font

for i in range(0, len(df_survey.columns)):
    # create a new figure
    plt.figure(figsize=(7, 7))
    # get the column name
    col_name = df_survey.columns[i]
    # replace Copilot in the column name with CodeRec
    #col_name = col_name.replace('Copilot', 'CodeRec')
    #col_name = col_name.replace('copilot', 'CodeRec')
    if col_name not in ['Which best describes your programming experience?', 'How proficient are you with Python?', 'How often do you use Copilot outside of today’s session?']:
        continue
    # get the column values
    col_values = df_survey[col_name].value_counts()
    # get the column values as a list
    col_values_list = col_values.tolist()
    # get the column index as a list
    col_index_list = col_values.index.tolist()
    # get the column index as a list
    col_index_list = [str(i) for i in col_index_list]
    # create horizontal bar chart with colors from bad to good but dont sort

    #plt.barh(col_index_list, col_values_list, color=['#ff0000', '#ff4000', '#ff8000', '#ffbf00', '#ffff00', '#bfff00', '#80ff00', '#40ff00', '#00ff00'])    
    plt.barh(col_index_list, col_values_list, height = 0.5)
    # set the title
    plt.title(col_name, fontsize = 20)
    # set the x label
    # set the y label
    plt.xlabel('Number of Participants', fontsize = 20)
    plt.yticks(fontsize=20)
# x ticks make x-large
    plt.xticks(fontsize=20)
    # save the figure
    plt.show()

# Average Statistics (Section 5.2)

In [ ]:
all_states = []
all_actions = []
all_times = []
all_states_continuous = []
all_states_list = []
all_times_list = []

all_times_continuous = []
for i in range(len(logs_by_user_session_labeled)):
    states = logs_by_user_session_labeled[i].LabeledState.to_numpy()
    actions = logs_by_user_session_labeled[i].StateName.to_numpy()
    times = logs_by_user_session_labeled[i].TimeSpentInState.to_numpy()
    # clean up states
    for j in range(len(states)):
        if states[j] == 'Edditing Last Suggestion (X)':
            states[j] = 'Editing Last Suggestion (X)'
        if states[j] == 'IDK':
            states[j] = 'IDK (I)'
        if states[j] == 'Looking up documentation' or states[j] == 'Looking up documentation (N)':
            states[j] = 'Looking up Documentation (N)'
        if states[j] == 'h':
            states[j] = 'Thinking About New Code To Write (F)'
        if states[j] == 'Thinking about new code to write (F)':
            states[j] = 'Thinking About New Code To Write (F)'

    # get indices where state is IDK
    idk_indices = [i for i in range(len(states)) if states[i] == 'IDK (I)']
    states = [states[i] for i in range(len(states)) if i not in idk_indices]
    actions = [actions[i] for i in range(len(actions)) if i not in idk_indices]
    times = [times[i] for i in range(len(times)) if i not in idk_indices]

    states_continuous = []
    times_continuous = []
    # merge states with same name
    last_state = states[0]
    times_cumalitive = 0
    for i in range(len(states)):
        if states[i] != last_state:
            states_continuous.append(last_state)
            last_state = states[i]
            times_continuous.append(times_cumalitive)
            times_cumalitive = times[i]
        else:
            times_cumalitive += times[i]
    states_continuous.append(last_state)
    times_continuous.append(times_cumalitive)
    
    all_states_list.extend(states)
    all_times_list.extend(times)
    all_states_continuous.extend(states_continuous)
    all_times_continuous.extend(times_continuous)
    all_states.append(states)
    all_actions.append(actions)
    all_times.append(times)


In [ ]:
# get average lenght of session
session_lengths = [ np.sum(times)/60 for times in all_times]
# avrage session length
np.mean(session_lengths)
# standard deviation of session length
np.std(session_lengths)
print(f'Average session length: {np.mean(session_lengths):.2f} +- {np.std(session_lengths):.2f}')
print(f' max lenght of session: {np.max(session_lengths):.2f} min lenght of session: {np.min(session_lengths):.2f}')

# number of events per session
session_counts_events = [len(times) for times in all_times]
print (f'Average number of events per session: {np.mean(session_counts_events):.2f} +- {np.std(session_counts_events):.2f}')
print(f'total number of events: {np.sum(session_counts_events)}')

In [ ]:
# unroll all actions into one list
all_actions_unrolled = []
for i in range(len(all_actions)):
    all_actions_unrolled.extend(all_actions[i])
# count how many shown events

# histogram of actions frequency
# counterr
import collections
counter = collections.Counter(all_actions_unrolled)
# normalize counter
counter_normalized = {k: v/sum(counter.values()) for k, v in counter.items()}
print(counter)
print(f' Accept Rate is {counter["Accepted"]/ counter["Shown"]}')
all_actions_binary = [] 
# for each list in all_actions, keep only Accepted and Rejected and convert to binary
for i in range(len(all_actions)):
    all_actions_binary.append([1 if action == 'Accepted' else 0 for action in all_actions[i] if action == 'Accepted' or action == 'Rejected'])
all_actions_binary = np.array(all_actions_binary)
# get mean for each list
all_actions_binary_mean = [np.mean(all_actions_binary[i]) for i in range(len(all_actions_binary))]
# get standard error
print(np.mean(all_actions_binary_mean))
print(np.std(all_actions_binary_mean))
print(np.median(all_actions_binary_mean))
print(np.min(all_actions_binary_mean))
print(np.max(all_actions_binary_mean))

# Use of Custom State (Section 5.2)

In [ ]:
all_states_clean = []
all_states_continuous = []
for i in range(len(logs_by_user_session_labeled)):
    states = logs_by_user_session_labeled[i].LabeledState.to_numpy()
    # clean up states
    for j in range(len(states)):
        if states[j] == 'Edditing Last Suggestion (X)':
            states[j] = 'Editing Last Suggestion (X)'
        if states[j] == 'IDK':
            states[j] = 'IDK (I)'
        if states[j] == 'Looking up documentation' or states[j] == 'Looking up documentation (N)':
            states[j] = 'Looking up Documentation (N)'
        if states[j] == 'h':
            states[j] = 'Thinking About New Code To Write (F)'
        if states[j] == 'Thinking about new code to write (F)':
            states[j] = 'Thinking About New Code To Write (F)'


    # get indices where state is IDK
    all_states_clean.append(states)
    all_states_continuous.extend(states)


In [ ]:
for i in range(21):
    counter = collections.Counter(all_states_clean[i])
    print(counter['IDK (I)'])
    


In [ ]:
# compare raw and cleaned states
import collections
counter = collections.Counter(all_states_continuous)
print(counter)
# get the total for counter
total = sum(counter.values())
print(total)

